In [1]:
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Dropout, Flatten, Embedding, Flatten, Dense, LSTM
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D
from tensorflow.keras.preprocessing import sequence
import tensorflow.keras.utils
import numpy as np
import pandas as pd
import spacy
import spacy_transformers
import en_trf_bertbaseuncased_lg
import os
import tqdm

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.dropna(inplace=True)

In [4]:
df_test = pd.read_csv('test.csv')

In [5]:
X = df['text']
y = df['sentiment']

In [6]:
y.replace({'neutral':0, 'negative':-1, 'positive':1}, inplace=True)

In [7]:
y = tensorflow.keras.utils.to_categorical(y, num_classes=3, dtype='float32')

In [8]:
from spacy_transformers import TransformersLanguage, TransformersWordPiecer, TransformersTok2Vec

name = "bert-base-uncased"
nlp = TransformersLanguage(trf_name=name, meta={"lang": "en"})
nlp.add_pipe(nlp.create_pipe("sentencizer"))
nlp.add_pipe(TransformersWordPiecer.from_pretrained(nlp.vocab, name))
nlp.add_pipe(TransformersTok2Vec.from_pretrained(nlp.vocab, name))
print(nlp.pipe_names)  # ['sentencizer', 'trf_wordpiecer', 'trf_tok2vec']

['sentencizer', 'trf_wordpiecer', 'trf_tok2vec']


In [12]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [20]:
# Print the original sentence.
print(' Original: ', X[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(X[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(X[0])))

# Print the sentence mapped to token ids.
# doc = nlp(tokenizer.tokenize(X[0]))
# word_id = doc._.trf_word_pieces
# print('Token IDs with wordpiecer: ',  word_id)

 Original:   I`d have responded, if I were going
Tokenized:  ['i', '`', 'd', 'have', 'responded', ',', 'if', 'i', 'were', 'going']
Token IDs:  [1045, 1036, 1040, 2031, 5838, 1010, 2065, 1045, 2020, 2183]


In [33]:
token=tokenizer.tokenize(X[2742])
print(token)
doc = nlp(X[2742])
word_id = doc._.trf_word_pieces
word_id

['fell', 'on', 'the', 'sidewalk', 'in', 'harvard', 'square', '.', 'stupid', 'cobb', '##les', '##tones', '!', 'now', 'my', 'hand', 'hurts']


[101,
 3062,
 2006,
 1996,
 11996,
 1999,
 5765,
 2675,
 1012,
 102,
 101,
 5236,
 17176,
 4244,
 11115,
 999,
 102,
 101,
 2085,
 2026,
 2192,
 13403,
 102]

In [32]:
X[2742]

'fell on the sidewalk in harvard square.  stupid cobblestones!  now my hand hurts'

In [30]:
model = spacy.load('en_trf_bertbaseuncased_lg')

In [40]:
max_lenght = len(max(word_id, key=len))

In [24]:
pd.DataFrame(word_id_2).to_csv("word_id.csv", index=None)

NameError: name 'word_id_2' is not defined

In [10]:
def clean_my_text(text):
    lemmatized = []
    text = text.lower()
    tokens = nlp(text)
    for word in tokens:
        if not word.is_stop and word.is_alpha:            
            lemmatized.append(word.lemma_)
    return lemmatized

In [29]:
def preproccess():
    clean_X = []
    for text in tqdm.tqdm(X):
        lemmatized = clean_my_text(text)
        if len(lemmatized) != 0:
            join_string = ' '.join(lemmatized)
            doc = nlp(join_string)
            word_id = doc._.trf_word_pieces
        else:
            word_id = lemmatized        
        clean_X.append(word_id)
    max_lenght = len(max(clean_X, key=len))    
    word_vec_X = sequence.pad_sequences(clean_X, maxlen = max_lenght, padding='pre')
    pd.DataFrame(word_vec_X).to_csv("word_vec_spacy_trans.csv", index=None)
    
    
        

In [30]:
preproccess()

100%|██████████| 27480/27480 [2:30:59<00:00,  3.03it/s]  


In [11]:
word_vec_X = pd.read_csv('word_id.csv')
word_vec_X = word_vec_X.to_numpy()
word_vec_X

array([[    0,     0,     0, ...,  2020,  2183,   102],
       [    0,     0,     0, ...,   999,   999,   102],
       [    0,     0,     0, ...,  1012,  1012,   102],
       ...,
       [    0,     0,     0, ..., 22038, 20348,   102],
       [    0,     0,     0, ...,  1008,  1012,   102],
       [    0,     0,     0, ...,  1007,  1007,   102]])

In [122]:
vocab_list = ['']
for text in clean_X:
    for word in text:        
        vocab_list.append(word)

# takes care of the duplicates
vocab_list = list(set(vocab_list))

In [17]:
list = [[1 ,2 ,3], [2], [2,2,2,2,2,2,2]]
len(max(list, key=len))

7

In [65]:
word_to_num = {}
num_to_word = {}
for i, word in enumerate(vocab_list):
    num_to_word[i] = word
    word_to_num[vocab_list[i]] = i

In [66]:
word_vec_x = [[word_to_num[word] for word in text] for text in clean_X]

In [67]:
word_vec_X = sequence.pad_sequences(word_vec_x, maxlen = 22, padding='pre' )

In [24]:
clean_x_series = pd.Series(clean_X)

In [25]:
clean_x_series = clean_x_series.apply(pd.Series)

In [26]:
clean_x_series.dropna(axis=0)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
1234,xo,hey,chick,u,alryt,u,dads,tmoro,sud,sumin,...,week,neva,dun,oot,week,lol,missed,ha,bye,hun
16149,av,ad,realy,gd,day,wiv,ciara,connolly,park,gate,...,new,brighton,fukin,funny,bt,sunburnt,luks,like,drivers,arm


In [71]:
pd.DataFrame(word_vec_X).to_csv("word_vec_X_trans.csv", index=None)

In [35]:
word_vec_X = pd.read_csv('word_id.csv')


In [36]:
word_vec_X = word_vec_X.to_numpy()

In [11]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(word_vec_X, y)

In [37]:
vocab_size = len(vocab_list) + 1 
vocab_size

NameError: name 'vocab_list' is not defined

In [44]:
model = Sequential()
model.add(Embedding(3400, 64, input_length=112))
model.add(Dropout(0.5))
model.add(BatchNormalization())
# model.add(LSTM(1024))
# model.add(Dropout(0.5))
model.add(LSTM(512, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(1, activation='sigmoid'))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.35))
model.add(BatchNormalization())

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.35))
model.add(BatchNormalization())

model.add(Dense(3, activation='softmax'))

In [45]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

In [46]:
model.fit(Xtrain, ytrain, epochs=3, batch_size=32, validation_split=0.2)

Train on 16488 samples, validate on 4122 samples
Epoch 1/3
   32/16488 [..............................] - ETA: 27:01

InvalidArgumentError:  indices[28,96] = 12954 is not in [0, 3400)
	 [[node sequential_2/embedding_2/embedding_lookup (defined at /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_10461]

Function call stack:
distributed_function


In [36]:
from sklearn.metrics import classification_report

y_pred = model.predict(Xtrain, batch_size=64, verbose=0)
y_pred_bool = np.argmax(y_pred, axis=1)
ytrain_bool = np.argmax(ytrain, axis=1)
print(classification_report(ytrain_bool, y_pred_bool))

              precision    recall  f1-score   support

           0       0.79      0.81      0.80      8362
           1       0.86      0.83      0.85      6477
           2       0.81      0.81      0.81      5771

    accuracy                           0.82     20610
   macro avg       0.82      0.82      0.82     20610
weighted avg       0.82      0.82      0.82     20610



In [37]:
y_pred = model.predict(Xtest, batch_size=64, verbose=0)
y_pred_bool = np.argmax(y_pred, axis=1)
ytest_bool = np.argmax(ytest, axis=1)
print(classification_report(ytest_bool, y_pred_bool))

              precision    recall  f1-score   support

           0       0.65      0.70      0.68      2755
           1       0.78      0.73      0.76      2105
           2       0.70      0.67      0.68      2010

    accuracy                           0.70      6870
   macro avg       0.71      0.70      0.71      6870
weighted avg       0.70      0.70      0.70      6870



### BERT

In [37]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(word_vec_X, y)

In [28]:
import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

In [29]:
model_dir = "./uncased_L-12_H-768_A-12"

bert_params = bert.params_from_pretrained_ckpt(model_dir)
l_bert = bert.BertModelLayer.from_params(bert_params, name="bert")

In [48]:
max_seq_len = 112
l_input_ids      = tensorflow.keras.layers.Input(shape=(max_seq_len,), dtype='int32')
# l_token_type_ids = tensorflow.keras.layers.Input(shape=(max_seq_len,), dtype='int32')

# using the default token_type/segment id 0
output = l_bert(l_input_ids) # output: [batch_size, max_seq_len, hidden_size]

cls_out = tensorflow.keras.layers.Lambda(lambda seq: seq[:, 0, :])(output)
cls_out = Dropout(0.5)(cls_out)
# cls_out = BatchNormalization()(cls_out)
logits = Dense(768, activation="tanh")(cls_out)
logits = Dropout(0.5)(logits)
# logits = Dense(128, activation="relu")(logits)
# logits = Dropout(0.5)(logits)
logits = Dense(units=3,activation="softmax")(logits)


model = tensorflow.keras.Model(inputs=l_input_ids, outputs=logits)
model.build(input_shape=(None, max_seq_len))


# model = tensorflowkeras.Model(inputs=l_input_ids, outputs=output)
# model.build(input_shape=(None, max_seq_len))

# provide a custom token_type/segment id as a layer input
# output = l_bert([l_input_ids, l_token_type_ids])          # [batch_size, max_seq_len, hidden_size]
# model = tensorflow.keras.Model(inputs=[l_input_ids, l_token_type_ids], outputs=output)
# model.build(input_shape=[(None, max_seq_len), (None, max_seq_len)])

In [49]:
bert_ckpt_file   = os.path.join(model_dir, "bert_model.ckpt")
bert.load_stock_weights(l_bert, bert_ckpt_file)

Done loading 196 BERT weights from: ./uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7f501408aeb8> (prefix:bert_2). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


[]

In [50]:
model.compile(optimizer=tensorflow.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics= ['accuracy'])

In [51]:
model.fit(Xtrain, ytrain, epochs=2, batch_size=8, validation_split=0.2)

Train on 16488 samples, validate on 4122 samples
Epoch 1/2
16488/16488 [==============================] - 868s 53ms/sample - loss: 0.4845 - accuracy: 0.7655 - val_loss: 0.3289 - val_accuracy: 0.8560
Epoch 2/2
16488/16488 [==============================] - 827s 50ms/sample - loss: 0.3247 - accuracy: 0.8620 - val_loss: 0.3504 - val_accuracy: 0.8594


In [53]:
from sklearn.metrics import classification_report

y_pred = model.predict(Xtrain, batch_size=64, verbose=0)
y_pred_bool = np.argmax(y_pred, axis=1)
ytrain_bool = np.argmax(ytrain, axis=1)
print(classification_report(ytrain_bool, y_pred_bool))

              precision    recall  f1-score   support

           0       0.82      0.87      0.85      8335
           1       0.91      0.87      0.89      6444
           2       0.89      0.85      0.87      5831

    accuracy                           0.86     20610
   macro avg       0.87      0.86      0.87     20610
weighted avg       0.87      0.86      0.86     20610



In [71]:
y_pred = loaded_model.predict(Xtest, batch_size=64, verbose=0)
y_pred_bool = np.argmax(y_pred, axis=1)
ytest_bool = np.argmax(ytest, axis=1)
print(classification_report(ytest_bool, y_pred_bool))

              precision    recall  f1-score   support

           0       0.74      0.79      0.76      2782
           1       0.84      0.81      0.83      2138
           2       0.81      0.77      0.79      1950

    accuracy                           0.79      6870
   macro avg       0.80      0.79      0.79      6870
weighted avg       0.79      0.79      0.79      6870



In [54]:
# serialize model to JSON
with open("model.json", "w") as json_file:
    json_file.write(model.to_json())

# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [68]:
model.save('model_save')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_save/assets


In [70]:
from tensorflow.keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json, custom_objects={"BertModelLayer": bert.BertModelLayer})
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


NameError: name 'loaded_model' is not defined